![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training Multilabel Classification Models with Legal NLP

In this notebook, you will learn how to use Spark NLP and Legal NLP to train multilabel classification models.

Let`s dive in!

In [0]:
from johnsnowlabs import nlp, legal, viz

from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

## Introduction

For the text classification tasks, we will use two annotators:

- `MultiClassifierDL`: `Multilabel Classification` (can predict more than one class for each text) using a Bidirectional GRU with Convolution architecture built with TensorFlow that supports up to 100 classes. The inputs are Sentence Embeddings such as state-of-the-art UniversalSentenceEncoder, BertSentenceEmbeddings or SentenceEmbeddings.
- `ClassifierDL`: uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. Then, a deep learning model (DNNs) built with TensorFlow that supports `Binary Classification` and `Multiclass Classification` (up to 100 classes).

The `ClassifierDLApproach` annotator trains a multiclass model, where the predictions is one category out of a predifined set of categories that are present in the training data.

## Loading the data

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/finance_clf_data.csv
dbutils.fs.cp("file:/databricks/driver/finance_clf_data.csv", "dbfs:/")

Out[3]: True

In [0]:
import pandas as pd
df = pd.read_csv('finance_clf_data.csv', encoding="utf8")
print(f"Shape of the full dataset: {df.shape}")

Shape of the full dataset: (6128, 3)

In [0]:
df.head()

Out[5]:

,text,label,len
0,Presently we do not believe any U S or State r...,business,402
1,\nnetwork outages or performance degradation ...,risk_factors,496
2,Available Information\nOur reports filed with ...,business,356
3,\n 42 530\n \n \n \n \n \n 42 530\nTotal liab...,financial_statements,359
4,8\nTable of Contents\ndevelopment employee eng...,business,582


In [0]:
df['label'].value_counts()

Out[6]: risk_factors 1926
financial_statements 1888
business 970
financial_conditions 346
form_10k_summary 240
executives_compensation 155
controls_procedures 138
equity 111
market_risk 100
executives 73
legal_proceedings 51
properties 48
security_ownership 46
exhibits 36
Name: label, dtype: int64

Since the deep learning models can take some time to train, we will limit our dataset to a smaller number of observations in order to illustrate how to use Spark NLP and Finance NLP annotators and pipelines to train the model, but without having to wait too much.

Please note that the quality and the quantity of training data is very relevant to the obtained trianed model, and the results we obtain here are for illustration purposes only. To obtain a more realistic model, pelase consider using the full dataset or addin extra observations from different sources.

In [0]:
from sklearn.model_selection import train_test_split

# The top 3 categories (number of observations)
filter_classes = ["risk_factors", "financial_statements", "business"]

# We make a random sample with 1000 observations
df = df.loc[df.label.isin(filter_classes)].sample(1000)

# Stratify split for train and test datasets
train_data, test_data = train_test_split(
    df, train_size=0.9, stratify=df.label, random_state=42
)

# Send to spark
train = spark.createDataFrame(train_data)
test = spark.createDataFrame(test_data)

In [0]:
train.groupBy("label").count().orderBy(F.col("count").desc()).show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 risk_factors| 365|
financial_statements| 356|
 business| 179|
+--------------------+-----+

In [0]:
test.groupBy("label").count().orderBy(F.col("count").desc()).show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 risk_factors| 41|
financial_statements| 39|
 business| 20|
+--------------------+-----+

## With Universal Encoder

In [0]:
%fs mkdirs file:/dbfs/multiclass_use

res0: Boolean = true

In [0]:
document_assembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")
)

embeddings = (
    nlp.UniversalSentenceEncoder.pretrained()
    .setInputCols("document")
    .setOutputCol("sentence_embeddings")
)

classsifierdl = (
    legal.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(30)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("file:/dbfs/multiclass_use")
    .setLr(0.001)
    .setBatchSize(4)
    .setDropout(0.15)
)

clf_pipeline = nlp.Pipeline(stages=[document_assembler, embeddings, classsifierdl])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 162 ms, sys: 51.9 ms, total: 214 ms
Wall time: 43.3 s

In [0]:
import os
log_file_name = os.listdir("/dbfs/multiclass_use")[0]

with open("/dbfs/multiclass_use/"+log_file_name, "r") as log_file :
    print(log_file.read())

Training started - epochs: 30 - learning_rate: 0.001 - batch_size: 4 - training_examples: 450 - classes: 3
Epoch 0/30 - 0.84s - loss: 108.63804 - acc: 0.60491073 - batches: 113
Epoch 1/30 - 0.44s - loss: 93.2874 - acc: 0.73883927 - batches: 113
Epoch 2/30 - 0.44s - loss: 85.61947 - acc: 0.84375 - batches: 113
Epoch 3/30 - 0.43s - loss: 81.13512 - acc: 0.87053573 - batches: 113
Epoch 4/30 - 0.42s - loss: 78.461784 - acc: 0.88616073 - batches: 113
Epoch 5/30 - 0.41s - loss: 76.84228 - acc: 0.8995536 - batches: 113
Epoch 6/30 - 0.42s - loss: 75.85659 - acc: 0.90401787 - batches: 113
Epoch 7/30 - 0.44s - loss: 75.21753 - acc: 0.91071427 - batches: 113
Epoch 8/30 - 0.46s - loss: 74.76673 - acc: 0.9129464 - batches: 113
Epoch 9/30 - 0.43s - loss: 74.406265 - acc: 0.9151786 - batches: 113
Epoch 10/30 - 0.43s - loss: 74.099884 - acc: 0.91741073 - batches: 113
Epoch 11/30 - 0.42s - loss: 73.82788 - acc: 0.91741073 - batches: 113
Epoch 12/30 - 0.44s - loss: 73.58166 - acc: 0.91964287 - batches: 113
Epoch 13/30 - 0.44s - loss: 73.363335 - acc: 0.91964287 - batches: 113
Epoch 14/30 - 0.46s - loss: 73.169525 - acc: 0.921875 - batches: 113
Epoch 15/30 - 0.44s - loss: 73.00098 - acc: 0.92410713 - batches: 113
Epoch 16/30 - 0.42s - loss: 72.852615 - acc: 0.92410713 - batches: 113
Epoch 17/30 - 0.45s - loss: 72.72597 - acc: 0.92633927 - batches: 113
Epoch 18/30 - 0.43s - loss: 72.61168 - acc: 0.92633927 - batches: 113
Epoch 19/30 - 0.45s - loss: 72.51238 - acc: 0.92633927 - batches: 113
Epoch 20/30 - 0.42s - loss: 72.420494 - acc: 0.92633927 - batches: 113
Epoch 21/30 - 0.42s - loss: 72.33612 - acc: 0.92633927 - batches: 113
Epoch 22/30 - 0.43s - loss: 72.25952 - acc: 0.92633927 - batches: 113
Epoch 23/30 - 0.42s - loss: 72.18505 - acc: 0.9285714 - batches: 113
Epoch 24/30 - 0.40s - loss: 72.11643 - acc: 0.9308036 - batches: 113
Epoch 25/30 - 0.40s - loss: 72.04982 - acc: 0.9308036 - batches: 113
Epoch 26/30 - 0.39s - loss: 71.98364 - acc: 0.9308036 - batches: 113
Epoch 27/30 - 0.41s - loss: 71.91841 - acc: 0.93303573 - batches: 113
Epoch 28/30 - 0.39s - loss: 71.85373 - acc: 0.93526787 - batches: 113
Epoch 29/30 - 0.42s - loss: 71.78748 - acc: 0.93526787 - batches: 113

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select("label", "text", "class.result").toPandas()
preds_df.head()

Out[14]:

,label,text,result
0,financial_statements,102\nSprout Social Inc \nNotes to Consolidated...,[financial_statements]
1,financial_statements,s RSM US LLP\nWe have served as the Company s...,[financial_statements]
2,business,\nPerfect Audience\n \nThe Perfect Audience p...,[business]
3,business,During each of the last few years sales of lic...,[risk_factors]
4,risk_factors,We currently serve our customers from third pa...,[risk_factors]


In [0]:
# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print(classification_report(preds_df["label"], preds_df["result"]))

precision recall f1-score support

 business 0.74 0.70 0.72 20
financial_statements 0.90 0.92 0.91 39
 risk_factors 0.83 0.83 0.83 41

 accuracy 0.84 100
 macro avg 0.82 0.82 0.82 100
 weighted avg 0.84 0.84 0.84 100

## With RoBerta Embeddings

In [0]:
%fs mkdirs file:/dbfs/multiclass_roberta

res3: Boolean = true

We do not have Legal Sentence Embeddings yet, But we can use the Legal RoBerta Embeddings and then average them.

In [0]:
document_assembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")
)

tokenizer = nlp.Tokenizer().setInputCols(["document"]).setOutputCol("token")

embeddings = (
    nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base", "en")
    .setInputCols(["document", "token"])
    .setOutputCol("embeddings")
    .setMaxSentenceLength(512)
)

embeddingsSentence = (
    nlp.SentenceEmbeddings()
    .setInputCols(["document", "embeddings"])
    .setOutputCol("sentence_embeddings")
    .setPoolingStrategy("AVERAGE")
)

classifierdl = (
    legal.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(3)
    .setLr(0.001)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("dbfs:/multiclass_roberta")
    .setBatchSize(4)
    .setDropout(0.15)
)

clf_pipeline_roberta = nlp.Pipeline(
    stages=[document_assembler, tokenizer, embeddings, embeddingsSentence, classifierdl]
)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]

In [0]:
%%time
clf_pipelineModel = clf_pipeline_roberta.fit(train)

CPU times: user 700 ms, sys: 237 ms, total: 937 ms
Wall time: 3min 6s

In [0]:
import os
log_files = os.listdir("/dbfs/multiclass_roberta")

with open("/dbfs/multiclass_roberta/"+log_files[0], "r") as log_file :
    print(log_file.read())

Training started - epochs: 3 - learning_rate: 0.001 - batch_size: 4 - training_examples: 900 - classes: 3
Epoch 0/3 - 1.22s - loss: 183.50458 - acc: 0.70666665 - batches: 225
Epoch 1/3 - 0.84s - loss: 164.02853 - acc: 0.79 - batches: 225
Epoch 2/3 - 0.79s - loss: 148.67624 - acc: 0.8744444 - batches: 225

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select("label", "text", "class.result").toPandas()

In [0]:
preds_df.head()

Out[38]:

,label,text,result
0,financial_statements,102\nSprout Social Inc \nNotes to Consolidated...,[financial_statements]
1,financial_statements,s RSM US LLP\nWe have served as the Company s...,[financial_statements]
2,business,\nPerfect Audience\n \nThe Perfect Audience p...,[business]
3,business,During each of the last few years sales of lic...,[risk_factors]
4,risk_factors,We currently serve our customers from third pa...,[risk_factors]


In [0]:
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))


precision recall f1-score support

 business 0.88 0.75 0.81 20
financial_statements 0.90 0.97 0.94 39
 risk_factors 0.90 0.90 0.90 41

 accuracy 0.90 100
 macro avg 0.90 0.88 0.88 100
 weighted avg 0.90 0.90 0.90 100

## Save model and Zip it for Modelshub Upload/Downloads

In [0]:
# Save a Spark NLP model
clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/ClfBert')